In [1]:
new_path_dataset ='/home/marrakchi/TFM/TFM_env/Datasets/ExKaldiSERDataset/'
path_dataset ='/home/marrakchi/TFM/TFM_env/Datasets/ExKaldiASRDatasetFormat/' 
ROOT_FOLDERS ='/home/marrakchi/TFM/TFM_env/Datasets/inter1sp/'

In [2]:
# Systems Libraries
import os

# For data processing
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display, HTML # For displaying tables
import string # for text processing
import IPython as ipd

# Import exkaldi package
import exkaldi
exkaldi.info.reset_kaldi_root('/kaldi')
from exkaldi import declare

# ExKaldi Configuration
ExKaldiInfo = exkaldi.info
print(ExKaldiInfo)
print(ExKaldiInfo.KALDI)
print(ExKaldiInfo.KALDI_ROOT)

exkaldi.info.reset_kaldi_root( yourPath )
If not, ERROR will occur when implementing some core functions.
ExKaldiInfo(version='1.3.5.4', major='1', minor='3', patch='5', upload='4')
Kaldi(version='5.5', major='5', minor='5')
/kaldi


# Data processing

In [6]:
# Preparing data structure for ExKaldi (WAV file, text file)

# Prepare ExKaldi files
transcription = exkaldi.Transcription(name="trans")
utt2spk = exkaldi.ListTable(name="utt2spk")
spk2utt = exkaldi.ListTable(name="spk2utt")
wavScp = exkaldi.ListTable(name="wavScp")

# Path
path_info = ROOT_FOLDERS+'index_sp/contents.lst'

# Read dataset transcriptions
X = pd.read_csv(path_info, sep="\t", header=0, usecols=[0,1,7], encoding='latin-1')

# Remove all rows with Utterance value missing
X.dropna(subset = ['LBO'], inplace=True)

# PREPROCESS TRANSCRIPTIONS
# Drop all containing digits:
X = X.drop(X[X.LBO.str.contains(r'\d+')].index)
# Label to lower case
X['LBO'] = X['LBO'].str.lower()
# Remove punctuations:
X['LBO'] = X['LBO'].str.replace('[{}]'.format(string.punctuation), '')
# Convert to plain text. Remove accents and any other symbols like ¨, æ, etc. 
X['LBO'] = X['LBO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Rewrite directory column
X['DIR'] = X.DIR.str.replace('\\', '/')
# Mix columns 1 & 2 into one
X['DIR'] = '/home/marrakchi/TFM/TFM_env/Datasets'+ X['DIR'].str.lower() + '/' + X['SRC'].str.lower()

# Remove SRC column
#X = X.drop(columns=['SRC'])

#Convert to dataframe format and drop extra columns
df = pd.DataFrame(X)
df = df.drop(columns=['DIR'])
df = df.reset_index(drop=True) # Refrech indexes

for num,row in enumerate (df.SRC):
    uttID = row.lower()
    speaker = uttID[4]
    if num == 0:
        print(uttID)
        print(speaker)
    # 1. text
    transcription[uttID] = df.LBO[num]
    # 4. wav.scp
    wavFilePath = os.path.join(new_path_dataset,'wav/',uttID +'.wav')
    wavScp[uttID] = wavFilePath
    # 2. utt2spk
    utt2spk[uttID] = speaker
    # 3. spk2utt
    if speaker not in spk2utt.keys():
        spk2utt[speaker] = f"{uttID}"
    else:
        spk2utt[speaker] += f" {uttID}"

# Save files
transcription.sort().save( os.path.join(new_path_dataset, "text") )
utt2spk.save( os.path.join(new_path_dataset, "utt2spk") )
spk2utt.save( os.path.join(new_path_dataset, "spk2utt") )  
wavScp.sort().save(os.path.join(new_path_dataset,"wav.scp") )

#Display
#display(HTML(df[0:10].to_html()))


spa1f001
f


'/home/marrakchi/TFM/TFM_env/Datasets/ExKaldiSERDataset/wav.scp'

In [4]:
# Create text file with transcriptions+accents
# Preparing data structure for ExKaldi (WAV file, text file)

# Prepare ExKaldi files
transcription = exkaldi.Transcription(name="trans")

# Path
path_info = ROOT_FOLDERS+'index_sp/contents.lst'

# Read dataset transcriptions
X = pd.read_csv(path_info, sep="\t", header=0, usecols=[0,1,7], encoding='latin-1')

# Remove all rows with Utterance value missing
X.dropna(subset = ['LBO'], inplace=True)

# PREPROCESS TRANSCRIPTIONS
# Drop all containing digits:
X = X.drop(X[X.LBO.str.contains(r'\d+')].index)
# Label to lower case
X['LBO'] = X['LBO'].str.lower()
# Remove punctuations:
X['LBO'] = X['LBO'].str.replace('[{}]'.format(string.punctuation), '')
# Convert to plain text. Remove accents and any other symbols like ¨, æ, etc.
#X['LBO'] = X['LBO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Rewrite directory column
X['DIR'] = X.DIR.str.replace('\\', '/')
# Mix columns 1 & 2 into one
X['DIR'] = '/home/marrakchi/TFM/TFM_env/Datasets'+ X['DIR'].str.lower() + '/' + X['SRC'].str.lower()

# Remove SRC column
#X = X.drop(columns=['SRC'])

#Convert to dataframe format and drop extra columns
df = pd.DataFrame(X)
df = df.drop(columns=['DIR'])
df = df.reset_index(drop=True) # Refrech indexes

for num,row in enumerate (df.SRC):
    uttID = row.lower()
    if num == 0:
        print(uttID)
    # 1. text
    transcription[uttID] = df.LBO[num]

# Save files
transcription.sort().save( os.path.join(new_path_dataset, "text+accents") )

#Display
display(HTML(df[0:10].to_html()))


spa1f001


,SRC,LBO
0,SPA1F001,con estoico respeto a la justicia adyacente guardó sus flechas
1,SPA1F002,fue inyectado en el abdomen y en una pierna
2,SPA1F003,la tensión volvió a aumentar el domingo
3,SPA1F004,cuando todavía eran baratos el vodka y el caviar
4,SPA1F005,sino el país mental de un patriota enloquecido
5,SPA1F006,abría sus puertas a estos flacos alumnos afroamericanos
6,SPA1F007,palestina seguía en estado paupérrimo después de su duunvirato
7,SPA1F008,el presidente de la federación portuguesa de fútbol
8,SPA1F009,bajo los efectos de la hipnosis tocaba la guzla perfectamente
9,SPA1F010,el oftalmólogo dijo que le hizo entrar en un proceso subfebril


In [46]:
# Getting all the audio paths

#Display
display(HTML(X[0:10].to_html()))

,DIR,SRC,LBO
0,/home/marrakchi/TFM/TFM_env/Datasets/inter1sp/f/sessa001/spa1f001,SPA1F001,con estoico respeto a la justicia adyacente guardo sus flechas
1,/home/marrakchi/TFM/TFM_env/Datasets/inter1sp/f/sessa001/spa1f002,SPA1F002,fue inyectado en el abdomen y en una pierna
2,/home/marrakchi/TFM/TFM_env/Datasets/inter1sp/f/sessa001/spa1f003,SPA1F003,la tension volvio a aumentar el domingo
3,/home/marrakchi/TFM/TFM_env/Datasets/inter1sp/f/sessa001/spa1f004,SPA1F004,cuando todavia eran baratos el vodka y el caviar
4,/home/marrakchi/TFM/TFM_env/Datasets/inter1sp/f/sessa001/spa1f005,SPA1F005,sino el pais mental de un patriota enloquecido
5,/home/marrakchi/TFM/TFM_env/Datasets/inter1sp/f/sessa001/spa1f006,SPA1F006,abria sus puertas a estos flacos alumnos afroamericanos
6,/home/marrakchi/TFM/TFM_env/Datasets/inter1sp/f/sessa001/spa1f007,SPA1F007,palestina seguia en estado pauperrimo despues de su duunvirato
7,/home/marrakchi/TFM/TFM_env/Datasets/inter1sp/f/sessa001/spa1f008,SPA1F008,el presidente de la federacion portuguesa de futbol
8,/home/marrakchi/TFM/TFM_env/Datasets/inter1sp/f/sessa001/spa1f009,SPA1F009,bajo los efectos de la hipnosis tocaba la guzla perfectamente
9,/home/marrakchi/TFM/TFM_env/Datasets/inter1sp/f/sessa001/spa1f010,SPA1F010,el oftalmologo dijo que le hizo entrar en un proceso subfebril


# Compute MFCC+CMVN

In [18]:
samp_path = new_path_dataset+'wav/spa1f131.wav'
ipd.display.Audio(filename=samp_path)

In [15]:
# Compute MFCC
feat = exkaldi.compute_mfcc(samp_path, name="mfcc")
print(feat)
# Compute spectrogram
# default args: frameWidth=25, frameShift=10, windowType='povey'
feat2 = exkaldi.compute_spectrogram(samp_path, rate=22050, name="spectrogram")
print(feat2)
# Feature dimensions
print(feat.dim)
print(feat2.dim)
# Feature utterances
print(feat.utts)
print(feat2.utts)

13
513
['spa1f001']
['spa1f001']


In [8]:
# Parameters
parallel = 1
mfccConfig={"--use-energy":"true"}
print(ExKaldiInfo.timeout)
ExKaldiInfo.set_timeout(28800) # 1/2 hour
print(ExKaldiInfo.timeout)

500
28800


In [9]:
feat = exkaldi.compute_mfcc(
                        os.path.join(new_path_dataset,"wav.scp"),
                        config=mfccConfig
                        #outFile=os.path.join(path_dataset,"mfcc","train","raw_mfcc.ark")
                    )
feat.save( os.path.join(new_path_dataset,"mfcc","raw_mfcc.ark") )
print(f"Generate SER_Dataset raw MFCC feature done.")

Generate SER_Dataset raw MFCC feature done.


In [10]:
# Compute CMVN
print('Start CMVN')
cmvn = exkaldi.compute_cmvn_stats(
                                feat=feat,
                                spk2utt=os.path.join(new_path_dataset,"spk2utt"),
                            )
cmvn.save( os.path.join(new_path_dataset,"mfcc","cmvn.ark") )
print(f"Generate CMVN statistics done.")
# Apply CMVN
feat = exkaldi.use_cmvn(
                    feat=feat,
                    cmvn=cmvn,
                    utt2spk=os.path.join(new_path_dataset,"utt2spk"),
                )
feat.save(os.path.join(new_path_dataset,"mfcc","mfcc_cmvn.ark"))
print(f"Generate MFCC feature (applied CMVN) done.")

Start CMVN
Generate CMVN statistics done.
Generate MFCC feature (applied CMVN) done.


# Test a GMM-HMM model with fmllr feature.
## --> Speaker Adapted Training (SAT)

In [6]:
# ARGS
delta = 2 #(n-order to add to the feature)
numIters = 35 # Training iterations
maxIterInc = 25 # Final iteration of increasing gaussians
realignIter = [10,20,30] # iteration to allign feature
order = 5 # N-GRAM to use (min 1 | max 6)
beam = 13 # Decode beam size
latBeam = 6 # Lattice beam size
acwt = 0.083333 # Acoustic model weight
parallel = 1 # parallel process to compute feature of train dataset (min 1 | max 10)
skipTrain = False # If True, skip training. Do decoding only
splice = 3 # How many left-right frames to splice
fmllrIter = [2,4,6,12] # The iteration to estimate fmllr matrix

In [7]:
declare.is_file( os.path.join(path_dataset,"train_sat","final.mdl") )
declare.is_file( os.path.join(path_dataset,"train_sat","tree") )

# Load tree
tree = exkaldi.load_tree( os.path.join(path_dataset,"train_sat","tree") )
# Load model
hmm = exkaldi.load_hmm( os.path.join(path_dataset,"train_sat","final.mdl") )
HCLGfile=os.path.join(path_dataset,"train_sat","graph",f"HCLG.{order}.fst")
tansformMatFile=os.path.join(path_dataset,"train_lda_mllt","trans.mat")
outDir=os.path.join(new_path_dataset,"train_sat",f"decode_{order}grams") # --> New Dataset path

# Load dictionary/lexicon
lexicons = exkaldi.load_lex(os.path.join(path_dataset,"dict","lexicons.lex"))

In [13]:
#print(f"Load test feature.")
#feat = exkaldi.load_feat(os.path.join(new_path_dataset,"mfcc","mfcc_cmvn.ark"))

print(f"Transform features")
if tansformMatFile is None:
    print("Feature type is delta. Add 2-order deltas.")
    feat = feat.add_delta(order=delta)
    feat = feat.save(os.path.join(outDir,"mfcc_cmvn_delta.ark"),returnIndexTable=True)
else:
    print("Feature type is lda+mllt")
    feat = feat.splice(left=splice,right=splice)
    print("Transform feature")
    feat = exkaldi.transform_feat(feat, tansformMatFile)
    feat = feat.save(os.path.join(outDir,"mfcc_cmvn_lda.ark"),returnIndexTable=True)

## 1. Estimate the primary transform matrix from alignment or lattice.
## We estimate it from lattice, so we decode it firstly.
print("Decode the first time with original feature.")
preLat = exkaldi.decode.wfst.gmm_decode(
                                    feat, 
                                    hmm, 
                                    HCLGfile, 
                                    symbolTable=lexicons("words"),
                                    beam=10, 
                                    latBeam=6, 
                                    acwt=acwt,
                                    maxActive=2000,
                                )
preLat.save(os.path.join(outDir,"test_premary.lat"))
preLat

print("Estimate the primary fMLLR transform matrix.")
preTransMatrix = exkaldi.hmm.estimate_fMLLR_matrix(
                                    aliOrLat=preLat,
                                    lexicons=lexicons,
                                    aliHmm=hmm, 
                                    feat=feat,
                                    adaHmm=None,
                                    silenceWeight=0.01,
                                    acwt=acwt,
                                    spk2utt=os.path.join(new_path_dataset,"spk2utt"),
                                )
del preLat
## 2. Transform feature. We will use new feature to estimate the secondary transform matrix from lattice.
print("Transform feature with primary matrix.")
fmllrFeat = exkaldi.use_fmllr(
                    feat,
                    preTransMatrix,
                    utt2spk=os.path.join(new_path_dataset,"utt2spk"),
                )
print("Decode the second time with primary fmllr feature.")
secLat = exkaldi.decode.wfst.gmm_decode(
                                    fmllrFeat, 
                                    hmm, 
                                    HCLGfile, 
                                    symbolTable=lexicons("words"),
                                    beam=beam,
                                    latBeam=latBeam,
                                    acwt=acwt,
                                    maxActive=7000,
                                    config={"--determinize-lattice":"false"},
                                )
print("Determinize secondary lattice.")
thiLat = secLat.determinize(acwt=acwt, beam=4)
print("Estimate the secondary fMLLR transform matrix.")
secTransMatrix = exkaldi.hmm.estimate_fMLLR_matrix(
                                    aliOrLat=thiLat,
                                    lexicons=lexicons,
                                    aliHmm=hmm, 
                                    feat=fmllrFeat,
                                    adaHmm=None,
                                    silenceWeight=0.01,
                                    acwt=acwt,
                                    spk2utt=os.path.join(new_path_dataset,"spk2utt"),
                                )
del fmllrFeat
del thiLat
## 3. Compose the primary matrix and secondary matrix and get the final transform matrix.
print("Compose the primary and secondary transform matrix.")
finalTransMatrix = exkaldi.hmm.compose_transform_matrixs(
                                                    matA=preTransMatrix,
                                                    matB=secTransMatrix,
                                                    bIsAffine=True,
                                                )
finalTransMatrix.save(os.path.join(outDir,"trans.ark"))
print("Transform feature with final matrix.")
## 4. Transform feature with the final transform matrix and use it to decode.
## We directly use the lattice generated in the second step. The final lattice is obtained.
finalFmllrFeat = exkaldi.use_fmllr(
                                feat,
                                finalTransMatrix,
                                utt2spk=os.path.join(new_path_dataset,"utt2spk"),
                            )
del finalTransMatrix
print("Rescore secondary lattice.")
lat = secLat.am_rescore(
                    hmm=hmm,
                    feat=finalFmllrFeat,
                )
print("Determinize secondary lattice.")
lat = lat.determinize(acwt=acwt, beam=6)
lat.save(os.path.join(outDir,"test.lat"))
#lat = exkaldi.load_lat(os.path.join(outDir,"test.lat"))
print("Generate lattice done.")

# TEXTO DE REFERENCIA (fonemas)
refText = exkaldi.load_transcription(os.path.join(new_path_dataset,"text"))
# Transform transcriptions from words to phones
word2phone = exkaldi.load_list_table(path_dataset+'word2phone.map')
refText = refText.convert(word2phone, unkSymbol='<UNK>')
print("Generate reference text done.")
refText.save(os.path.join(outDir,"ref.txt") )

# Penalty: 0.0, LMWT: 10, PER: 22.74%

print("Now score:")
bestWER = (1000, 0, 0)
bestResult = None

for penalty in [0., 0.5, 1.0]:
    for LMWT in range(1, 11):
        # Add penalty
        newLat = lat.add_penalty(penalty)

        # RESULTADOS PREDECIDOS (int)
        # Get 1-best result (phone-level)
        result = newLat.get_1best(lexicons("phones"), hmm, lmwt=LMWT, acwt=1, phoneLevel=True)

        # RESULTADOS PREDECIDOS (fonemas)
        # Transform from int value format to text format
        result = exkaldi.hmm.transcription_from_int(result, lexicons("phones"))

        # Compute PER
        score = exkaldi.decode.score.wer(ref=refText, hyp=result, mode="present")
        
        if score.WER < bestWER[0]:
            bestResult = result
            bestWER = (score.WER, penalty, LMWT)
        print(f"Penalty: {penalty}, LMWT: {LMWT}, PER: {score.WER}%")
        

print("Score done. Save the best result.")
bestResult.save(os.path.join(outDir, "hyp.txt") )

with open(os.path.join(outDir,"best_PER"),"w") as fw:
    fw.write( f"PER {bestWER[0]}, penalty {bestWER[1]}, LMWT {bestWER[2]}" )

Transform features
Feature type is lda+mllt
Transform feature
Decode the first time with original feature.
Estimate the primary fMLLR transform matrix.
Transform feature with primary matrix.
Decode the second time with primary fmllr feature.
Determinize secondary lattice.
Estimate the secondary fMLLR transform matrix.
Compose the primary and secondary transform matrix.
Transform feature with final matrix.
Rescore secondary lattice.
Determinize secondary lattice.
Generate lattice done.
Generate reference text done.
Now score:
Penalty: 0.0, LMWT: 1, PER: 36.71%
Penalty: 0.0, LMWT: 2, PER: 36.24%
Penalty: 0.0, LMWT: 3, PER: 35.79%
Penalty: 0.0, LMWT: 4, PER: 35.33%
Penalty: 0.0, LMWT: 5, PER: 34.9%
Penalty: 0.0, LMWT: 6, PER: 34.45%
Penalty: 0.0, LMWT: 7, PER: 34.15%
Penalty: 0.0, LMWT: 8, PER: 33.84%
Penalty: 0.0, LMWT: 9, PER: 33.66%
Penalty: 0.0, LMWT: 10, PER: 33.5%
Penalty: 0.5, LMWT: 1, PER: 36.69%
Penalty: 0.5, LMWT: 2, PER: 36.2%
Penalty: 0.5, LMWT: 3, PER: 35.72%
Penalty: 0.5, LM

In [25]:
lat = exkaldi.load_lat(os.path.join(outDir,"test.lat"))

# TEXTO DE REFERENCIA (palabras)
refText = exkaldi.load_transcription(os.path.join(new_path_dataset,"text+accents"))
print(refText.subset(nHead=1))
#refText.save(os.path.join(outDir,"ref_words_accents.txt"))

print("Now score:")
#bestWER = (1000, 0, 0)
#bestResult = None

penalty = 1
LMWT = 10

#for penalty in [0., 0.5, 1.0]:
#    for LMWT in range(1, 11):
# Add penalty
#print(penalty)
#print(LMWT)
newLat = lat.add_penalty(penalty)
#print(newLat)
# RESULTADOS PREDECIDOS (int)
# Get 1-best result (phone-level)
result = newLat.get_1best(lexicons("words"), hmm, lmwt=LMWT, acwt=1, phoneLevel=False)
print(result.subset(nHead=1))

# RESULTADOS PREDECIDOS (palabras)
# Transform from int value format to text format
result = exkaldi.hmm.transcription_from_int(result, lexicons("words"))
print(result.subset(nHead=1))

# Compute PER
score = exkaldi.decode.score.wer(ref=refText.subset(nHead=1), hyp=result.subset(nHead=1), mode="present")
print(score)

#if score.WER < bestWER[0]:
#    bestResult = result
#    bestWER = (score.WER, penalty, LMWT)
#print(f"Penalty: {penalty}, LMWT: {LMWT}, WER: {score.WER}%")
        
#print("Score done. Save the best result.")
#bestResult.save(os.path.join(outDir, "hyp_words.txt") )
#with open(os.path.join(outDir,"best_WER"),"w") as fw:
#    fw.write( f"WER {bestWER[0]}, penalty {bestWER[1]}, LMWT {bestWER[2]}" )

{'spa1f001': 'con estoico respeto a la justicia adyacente guardó sus flechas'}
Now score:
{'spa1f001': '19428 36960 75793 51942 51543 24779 43699 83006 40444'}
{'spa1f001': 'con estoico respeto la justicia decente guardo sus flechas'}
Score(WER=30.0, words=10, insErr=0, delErr=1, subErr=2, SER=100.0, sentences=1, wrongSentences=1, missedSentences=0)


In [24]:
print(score.WER)

54.63


In [15]:
#lat = exkaldi.load_lat(os.path.join(outDir,"test.lat"))

# TEXTO DE REFERENCIA (palabras)
refText = exkaldi.load_transcription(os.path.join(new_path_dataset,"text"))
refText.save(os.path.join(outDir,"ref_words.txt"))

print("Now score:")
bestWER = (1000, 0, 0)
bestResult = None

#penalty = 1
#LMWT = 10

for penalty in [0., 0.5, 1.0]:
    for LMWT in range(1, 11):
        # Add penalty
        #print(penalty)
        #print(LMWT)
        newLat = lat.add_penalty(penalty)
        #print(newLat)
        # RESULTADOS PREDECIDOS (int)
        # Get 1-best result (phone-level)
        result = newLat.get_1best(lexicons("words"), hmm, lmwt=LMWT, acwt=1, phoneLevel=False)
        #print(result.subset(nHead=1))

        # RESULTADOS PREDECIDOS (palabras)
        # Transform from int value format to text format
        result = exkaldi.hmm.transcription_from_int(result, lexicons("words"))
        #print(result.subset(nHead=1))

        # Compute PER
        score = exkaldi.decode.score.wer(ref=refText, hyp=result, mode="present")
        #print(score)

        if score.WER < bestWER[0]:
            bestResult = result
            bestWER = (score.WER, penalty, LMWT)
        print(f"Penalty: {penalty}, LMWT: {LMWT}, WER: {score.WER}%")
        
print("Score done. Save the best result.")
bestResult.save(os.path.join(outDir, "hyp_words.txt") )
with open(os.path.join(outDir,"best_WER"),"w") as fw:
    fw.write( f"WER {bestWER[0]}, penalty {bestWER[1]}, LMWT {bestWER[2]}" )

Now score:
Penalty: 0.0, LMWT: 1, WER: 69.56%
Penalty: 0.0, LMWT: 2, WER: 67.56%
Penalty: 0.0, LMWT: 3, WER: 65.5%
Penalty: 0.0, LMWT: 4, WER: 63.39%
Penalty: 0.0, LMWT: 5, WER: 61.22%
Penalty: 0.0, LMWT: 6, WER: 58.84%
Penalty: 0.0, LMWT: 7, WER: 56.86%
Penalty: 0.0, LMWT: 8, WER: 54.78%
Penalty: 0.0, LMWT: 9, WER: 53.02%
Penalty: 0.0, LMWT: 10, WER: 51.72%
Penalty: 0.5, LMWT: 1, WER: 69.29%
Penalty: 0.5, LMWT: 2, WER: 67.07%
Penalty: 0.5, LMWT: 3, WER: 64.83%
Penalty: 0.5, LMWT: 4, WER: 62.56%
Penalty: 0.5, LMWT: 5, WER: 60.05%
Penalty: 0.5, LMWT: 6, WER: 57.67%
Penalty: 0.5, LMWT: 7, WER: 55.56%
Penalty: 0.5, LMWT: 8, WER: 53.53%
Penalty: 0.5, LMWT: 9, WER: 51.98%
Penalty: 0.5, LMWT: 10, WER: 50.65%
Penalty: 1.0, LMWT: 1, WER: 69.05%
Penalty: 1.0, LMWT: 2, WER: 66.72%
Penalty: 1.0, LMWT: 3, WER: 64.21%
Penalty: 1.0, LMWT: 4, WER: 61.78%
Penalty: 1.0, LMWT: 5, WER: 59.11%
Penalty: 1.0, LMWT: 6, WER: 56.72%
Penalty: 1.0, LMWT: 7, WER: 54.6%
Penalty: 1.0, LMWT: 8, WER: 52.58%
Penalty: 